In [2]:
import pandas as pd
df = pd.read_csv('events.csv')

In [7]:
df.groupby('sessionId').count()

,Unnamed: 0,artist,song,duration,ts,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
sessionId,,,,,,,,,,,,,,,,,,,
221,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
295,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
319,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
598,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
747,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38275,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
38282,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
38386,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [20]:
df

,Unnamed: 0,artist,song,duration,ts,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
0,0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.373463,35.380939,9783,Parker,Haley,F,1616086800000
1,1,Gershon Kingsley,Trumansburg Whistle,180.08770,1616086803000,33160,Logged In,paid,6,Cincinnati,45238,OH,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,-84.610282,39.107658,16205,Smith,Jordan,F,1616086800000
2,2,AUDIO.3,SPONT.AN,241.44934,1616086804000,16311,Logged In,paid,12,Pecos,79772,TX,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-103.627309,31.422363,16312,Flores,Kylen,M,1616086800000
3,3,Mario Piu,Communication,430.54974,1616086805000,6939,Logged In,free,0,La Puente,91744,CA,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,-117.936470,34.031415,6940,Nelson,Caedyn,F,1616086800000
4,4,Boyz II Men,In My Life,151.58812,1616086806000,1095,Logged In,paid,7,Morrisonville,62546,IL,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-89.445923,39.431455,1096,Gonzalez,Emma,F,1616086800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,John Mayall,Don't Waste My Time,193.12280,1616088081000,37522,Logged In,free,0,Hackensack,7601,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.045698,40.889398,7139,Parks,Jesus,M,1616086800000
996,996,Solas,Clothes Of Sand,255.97342,1616088081000,32671,Logged In,paid,7,York,36925,AL,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-88.183591,32.432254,5102,Reed,Oliver,M,1616086800000
997,997,Humble Pie,Intro/Whatcha Gonna Do about It,288.02567,1616088085000,7118,Logged In,paid,12,Manchester,6040,CT,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,-72.525608,41.761415,7119,Smith,Melody,F,1616086800000
998,998,Bryan White,I Can't Wait 'Til Christmas (LP Version),240.90077,1616088087000,29636,Logged In,paid,49,Salt Lake City,84115,UT,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.892687,40.714497,2920,Perez,Genevieve,F,1616086800000


In [19]:
df.query('userId==748 and itemInSession==30').sort_values('ts', ascending=True).head(10)

,Unnamed: 0,artist,song,duration,ts,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
650,650,Irma Thomas,The Same Love That Made Me Laugh [LP Version],195.36934,1616087608000,747,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000
